In [ ]:
import math
import numpy as np
from IPython.display import clear_output
from tqdm import tqdm_notebook as tqdm

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

import matplotlib as mpl
import matplotlib.cm as cm

import torch
from torch import Tensor
from torch import nn
from torch.nn  import functional as F
from torch.autograd import Variable
import random
import torchvision
import torchvision.transforms as transforms
from source.utils import Class_Dataset
from tqdm import tqdm
from source.neural_integral_operator import integral_operator
from source.models import G_global
from source.integrators import MonteCarlo

use_cuda = torch.cuda.is_available()

In [ ]:
if torch.cuda.is_available():  
    device = "cuda:0" 
else:  
    device = "cpu"

In [ ]:
dim = 3
batch_size = 4
mc = MonteCarlo()
mc_samplings=2000
enc_dim = 8

In [ ]:
encoder = torch.nn.Conv1d(
    1,
    enc_dim+1,
    [16],
    stride=8,
    ).to(device)

In [ ]:
shapes = encoder(torch.rand(batch_size,400,1).permute(0,2,1).to(device)).shape

In [ ]:
shapes

In [ ]:
times = torch.linspace(0,1,shapes[2]).to(device)

In [ ]:
G = G_global(enc_dim,dim,[16,16]).to(device)

In [ ]:
model = integral_operator(dim, times, G, mc, num_internal_points=times.shape[0],mc_samplings=mc_samplings).to(device)

In [ ]:
import mat73

In [ ]:
X = mat73.loadmat('XMeatData.mat')

In [ ]:
X

In [ ]:
XCA, XCB, XPA, XPB, XTA, XTB = torch.from_numpy(X['XChickenA']).unsqueeze(-1), torch.from_numpy(X['XChickenB']).unsqueeze(-1), torch.from_numpy(X['XPorkA']).unsqueeze(-1), torch.from_numpy(X['XPorkB']).unsqueeze(-1), torch.from_numpy(X['XTurkeyA']).unsqueeze(-1), torch.from_numpy(X['XTurkeyB']).unsqueeze(-1)

In [ ]:
XCA.shape, XCB.shape, XPA.shape, XPB.shape, XTA.shape, XTB.shape

In [ ]:
Data = torch.cat([XCA, XCB, XPA, XPB, XTA, XTB], dim=0).to(device)

In [ ]:
Data.shape

In [ ]:
Data = Data[:,:400,:]

In [ ]:
#Data = torch.nn.functional.normalize(Data,p=1053)

In [ ]:
# mean = Data.mean(dim=0)
# std = Data.std(dim=0)

# # Standardize the data
# standardized_data = (Data - mean) / std

In [ ]:
# Calculate min and max for each feature
data_min = Data.min(dim=0)[0]
data_max = Data.max(dim=0)[0]

# Min-Max Scaling
Data = (Data - data_min) / (data_max - data_min)

In [ ]:
#Visualize
for i in range(Data.shape[0]):
    plt.plot(Data[i,:,0].cpu())
    plt.show

In [ ]:
y.shape

In [ ]:
Data.shape

In [ ]:
ClsC, ClsP, ClsT = torch.zeros(40,1), torch.zeros(40,1)+1, torch.zeros(40,1)+2

In [ ]:
Cls = torch.cat([ClsC, ClsP, ClsT]).to(device)

In [ ]:
permutation = torch.randperm(Data.shape[0])
print(permutation)

In [ ]:
Data = Data[permutation,...]
Cls = Cls[permutation,...]

In [ ]:
Data_train = Data[:-40,...].to(device)
Cls_train = Cls[:-40,...].to(device)
Data_val = Data[-40:-20,...].to(device)
Cls_val = Cls[-40:-20,...].to(device)
    
Data_test = Data[-20:,...].to(device)
Cls_test = Cls[-20:,...].to(device)

In [ ]:
Cls_train = Cls_train.type(torch.LongTensor).squeeze()
Cls_val = Cls_val.type(torch.LongTensor).squeeze()
Cls_test = Cls_test.type(torch.LongTensor).squeeze()

In [ ]:

train_loader = torch.utils.data.DataLoader(
                                   Class_Dataset(Data_train, Cls_train),
                                   batch_size=batch_size, 
                                   shuffle=True
)
valid_loader = torch.utils.data.DataLoader(
                                   Class_Dataset(Data_val, Cls_val),
                                   batch_size=batch_size, 
                                   shuffle=False
)

test_loader = torch.utils.data.DataLoader(
                                   Class_Dataset(Data_test, Cls_test), 
                                   batch_size=1,
                                   shuffle=False
)


# if args.mode == 'train':
#     dataloaders = {'train': train_loader,
#                    'val': valid_loader,
#                   }
# else:
#     dataloaders = {
#                    'test': test_loader,
#                   }

In [ ]:
torch.save(test_loader,'test_loader_meat')

In [ ]:
def flatten_kernel_parameters(kernel):
    p_shapes = []
    flat_parameters = []
    for p in kernel.parameters():
        p_shapes.append(p.size())
        flat_parameters.append(p.flatten())
    return torch.cat(flat_parameters)

In [ ]:
flatten_kernel_parameters(model).shape[0] + flatten_kernel_parameters(encoder).shape[0]

In [ ]:
patience = 50
factor = 0.5

In [ ]:
All_parameters = list(model.parameters()) + list(encoder.parameters())
optimizer = torch.optim.Adam(All_parameters, lr=1e-3, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
scheduler = \
torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=101, eta_min=1e-7)
#torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=patience, min_lr=1e-9,factor=factor)
warmup = 7

In [ ]:
epochs = 3000

In [ ]:
len(train_loader)

In [ ]:
verbose = False
losses = []
best_loss = np.inf
all_val_loss = []

for i in range(epochs):
    print("Epoch:", i)
    
    tot_loss = 0.
    counter = 0
    for  spectra, labels in tqdm(train_loader):

        if verbose: print(spectra.shape)
        spectra = spectra
        if verbose: print(spectra.shape)
        
        encoded = encoder(spectra.permute(0,2,1)).permute(0,2,1)
        
        inputs, x = encoded[...,:-1], encoded[:,-1,-1:]
        
        outputs = model(inputs,x)
        
        loss = criterion(outputs, labels.to(device))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        tot_loss += loss.item()
        counter += 1

        if verbose and i % 20 == 0:
            print(loss.item())
    
    losses.append(tot_loss/counter)


    with torch.no_grad():
        counter = 0
        val_loss = 0.
    for  spectra, labels in tqdm(valid_loader):

        if verbose: print(spectra.shape)
        spectra = spectra
        if verbose: print(spectra.shape)

        encoded = encoder(spectra.permute(0,2,1)).permute(0,2,1)
        inputs, x = encoded[...,:-1], encoded[:,-1,-1:]
        outputs = model(inputs,x)
        
        loss_validation = criterion(outputs, labels.to(device))

        counter += 1
        val_loss += loss_validation.item()
        

    if i > warmup:
        scheduler.step(val_loss/counter)
    all_val_loss.append(val_loss/counter)  

    plt.figure(0, figsize=(8,8),facecolor='w')           
    plt.plot(np.log10(losses),label='Train loss')
    plt.plot(np.log10(all_val_loss),label='Val loss')
    plt.xlabel("Epoch")
    plt.ylabel("MSE Loss")
    plt.legend()
    plt.savefig('losses')
    plt.close()


    if all_val_loss[-1] < best_loss:
        print('Saving for epoch: ', i, all_val_loss[-1])
        torch.save(G,'kernel')
        torch.save(encoder,'encoder')
        best_loss = all_val_loss[-1]

In [ ]:
test_loader = torch.load('test_loader_meat')

In [ ]:
len(test_loader)

In [ ]:
G = torch.load('kernel').to(device)
encoder = torch.load('encoder').to(device)

In [ ]:
model = integral_operator(dim, times, G, mc, num_internal_points=times.shape[0],mc_samplings=5000).to(device)

In [ ]:
tot_predicted = torch.tensor([]).to(device)
tot_labels = torch.tensor([]).to(device)
tot_outputs = torch.tensor([]).to(device)
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for spectra, labels in tqdm(test_loader):
        spectra = spectra

        encoded = encoder(spectra.permute(0,2,1)).permute(0,2,1)
        inputs, x = encoded[...,:-1], encoded[:,-1,-1:]
        outputs = model(inputs,x)
        tot_outputs = torch.cat([tot_outputs,outputs],0)
        
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels.to(device)).sum().item() 
        
        tot_predicted = torch.cat([tot_predicted,predicted], dim=0)
        tot_labels = torch.cat([tot_labels, labels.to(device)], dim=0)

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy: {acc} %')

In [ ]:
softmax = torch.nn.Softmax(dim=1)

In [ ]:
softmax(tot_outputs)

In [ ]:
from sklearn.metrics import classification_report
classification_report(tot_labels.cpu(),tot_predicted.cpu())